In [1]:
import pyw_hnswlib as hnswlib
import numpy as np
from sentence_transformers import SentenceTransformer
import threading
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import re
import string
import plotly.express as px

import ipywidgets as widgets
from IPython.display import display
import imgkit
import os
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
hindi_range = range(int("0900", 16), int("097F", 16))
english_range = range(int("0061", 16), int("007A", 16) + 1)
punjabi_range = range(int("0A00", 16), int("0A7F", 16))
gujarati_range = range(int("0A80", 16), int("0AFF", 16))
telugu_range = range(int("0C00", 16), int("0C7F", 16))
tamil_range = range(int("0B80", 16), int("0BFF", 16))
kannada_range = range(int("0C80", 16), int("0CFF", 16))
odia_range = range(int("0B00", 16), int("0B7F", 16))
bengali_range = range(int("0980", 16), int("09FF", 16))

# define a function to detect the language of a sentence
def detect_language(sentence):
    # count the number of characters in each language range
    hindi_count = sum(1 for c in sentence if ord(c) in hindi_range)
    english_count = sum(1 for c in sentence if ord(c) in english_range)
    punjabi_count = sum(1 for c in sentence if ord(c) in punjabi_range)
    gujarati_count = sum(1 for c in sentence if ord(c) in gujarati_range)
    telugu_count = sum(1 for c in sentence if ord(c) in telugu_range)
    tamil_count = sum(1 for c in sentence if ord(c) in tamil_range)
    kannada_count = sum(1 for c in sentence if ord(c) in kannada_range)
    odia_count = sum(1 for c in sentence if ord(c) in odia_range)
    bengali_count = sum(1 for c in sentence if ord(c) in bengali_range)

    # determine the language with the highest character count
    language_counts = {
        "Hindi": hindi_count,
        "English": english_count,
        "Punjabi": punjabi_count,
        "Gujarati": gujarati_count,
        "Telugu": telugu_count,
        "Tamil": tamil_count,
        "Kannada": kannada_count,
        "Odia": odia_count,
        "Bengali": bengali_count
    }
    max_count = max(language_counts.values())
    language = [k for k, v in language_counts.items() if v == max_count][0]

    return language



In [39]:
index_file_path = "/home/installer/ps/long_term/my_index"


# Load index from binary file
index = hnswlib.Index(space='l2', dim=768)
index.load_index(index_file_path)
index.set_ef(1000000)


In [4]:
import sys
size = sys.getsizeof(index)
size

48

In [5]:

# device = "cpu"
# from sentence_transformers import SentenceTransformer

model_salesken = SentenceTransformer("salesken/similarity-eng-hin_latin")


In [6]:
# # Load the model from a pickle file
# with open('model_salesken.pkl', 'rb') as f:
#     model_salesken = pickle.load(f)

In [7]:

# # Save the model to a pickle file
# with open('model_salesken.pkl', 'wb') as f:
#     pickle.dump(model_salesken, f)

In [8]:
# r = pd.read_csv('data/chunk0.csv',  header = None, nrows = 1, low_memory=False)

In [9]:
# Load the pandas dataframe
df1 = pd.read_csv('data/chunk0.csv', usecols= [0, 21],  header = None, low_memory=False)
df2 = pd.read_csv('data/chunk1.csv',  usecols= [0, 21], header = None, low_memory=False)
df3 = pd.read_csv('data/chunk2.csv',  usecols= [0, 21], header = None, low_memory=False)
df4 = pd.read_csv('data/chunk3.csv',  usecols= [0, 21], nrows = 800000, header = None, low_memory=False)
df = pd.concat([df1, df2, df3, df4])




In [10]:

# df = df[[0,21]]
df.columns = ['reg_no', 'subject_content']

In [11]:
df.reset_index(drop=True, inplace = True)
df

,reg_no,subject_content
0,-- Se/E/2015/00001,my grievance i am uploading in pdf format
1,-- Se/E/2015/00002,This is in regards to Vendor Registration Appl...
2,--SEL/E/2015/00001,hjjkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkkk...
3,--SEL/E/2015/00002,"Respected Sir, SUB :Telangana State - ANTI COR..."
4,--SEL/E/2015/00003,This complaint is regarding FIR no. 347 filled...
...,...,...
3799998,PMOPG/D/2016/0263471,FRANK
3799999,PMOPG/D/2016/0263477,NR/RAHU
3800000,PMOPG/D/2016/0263479,PG/SAT
3800001,PMOPG/D/2016/0263483,FRANK


In [12]:
with open("df.pkl", "wb") as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
# df.subject_content.to_csv("tmp.csv")

In [14]:

# Function to split text into lines with maximum words_per_line words per line
def split_text_to_lines(text, words_per_line=20):
    words = text.split()
    lines = []
    line = []
    for word in words:
        line.append(word)
        if len(line) == words_per_line:
            lines.append(' '.join(line))
            line = []
    if line:
        lines.append(' '.join(line))
    return '\n'.join(lines)



In [15]:
with open('final_wording.pkl', 'rb') as f:
    final_wording = pickle.load(f)

In [16]:

def clean_text(text):
    """
    Preprocess textual data by performing the following steps:
    1. Remove punctuation
    2. Convert text to lowercase
    3. Remove digits and special characters
    4. Remove extra whitespaces
    """
    if not isinstance(text, (str, bytes)):
        return text
    
    # remove links
    text = re.sub(r'http\S+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove digits and special characters
    text = re.sub(r'\d+', '', text)
    
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Correct Roman words using the predefined dictionary
    words = text.split()
    for i in range(len(words)):
        if words[i] in final_wording:
            words[i] = final_wording[words[i]]
    text = ' '.join(words)
    
    return text

In [17]:
# index.set_num_threads(8)

In [18]:
# create a lock
lock = threading.Lock()

# function for performing KNN search
def knn_search(index, query_vec, kk):
    lock.acquire() # acquire the lock before accessing the index
    labels, distances = index.knn_query(query_vec, k=kk)
    lock.release() # release the lock after the search is complete
    return labels, distances

In [61]:
def process_input_string(query):
    query_embedding = model_salesken.encode(clean_text(query))
    labels, distances = knn_search(index, query_embedding, kk=100)
    
    t = dict()
    for l,d in zip(labels[0], distances[0]):
        t[l]=d
    print(t)
        
    out  = df[df.reg_no.isin(labels[0])]
    print(len(out))
    out["subject_content_cleaned"] = out.subject_content.apply(clean_text)
    
        # remove empty sentences or sentences with just spaces
    # out = out.dropna(subset=["subject_content_cleaned"])
    # out = out[out["subject_content_cleaned"].str.strip().astype(bool)]
    out.reset_index(drop=True, inplace = True)
    out["Language"] = out['subject_content_cleaned'].apply(detect_language)
    print(str(out.Language.value_counts()))
    print()
    out.drop(["subject_content_cleaned"], axis=1, inplace=True)

    

    # for col in out.columns:
    out["subject_content"] = out["subject_content"].apply(split_text_to_lines)
    # Add distances as a new column
    print(out.columns)
    print(out["reg_no"].values)
    out['distances'] = out["reg_no"].apply(lambda x: t.get(x, -1))
    
    
    # out = out[out.distances < 10.0]
    
    # group the dataframe by language
    grouped = out.groupby('Language')

    # create a new empty dataframe to store the modified output
    new_out = pd.DataFrame()

    # iterate over each language group
    for lang, group in grouped:
        # get the first two rows of the group
        sample = group.head(3)
        # add the sample to the new dataframe
        new_out = pd.concat([new_out, sample])

    # reset the index of the new dataframe
    new_out.reset_index(drop=True, inplace=True)
    

    # replace the original dataframe with the modified one
    out = new_out
    
    out = out.sort_values(by='distances', ascending=True)
    
    # convert the dataframe to an HTML table
    html_table = out.to_html()

    # use imgkit to convert the HTML table to an image
    imgkit.from_string(html_table, "sample_images/"+query+".png")
    # out.to_csv(, index=False, lineterminator='\r\n\n')
    # return out

    del query_embedding
    del labels
    del distances
    del out
    del grouped
    del new_out
    del html_table

    
    # return out
    
    

In [20]:
poverty_issues = ['Lack of access to basic necessities such as food and shelter.',
                  'Poor nutrition and health outcomes for impoverished individuals.',
                  'Limited economic opportunities for impoverished individuals and communities.',
                  'Rural poverty and lack of development in many areas.',
                  'High levels of poverty among marginalized and disadvantaged groups.',
                  'Lack of social safety net and government support for impoverished individuals.',
                  'Challenges in accessing education and healthcare for impoverished individuals.',
                  'Child poverty and exploitation, including child labor and trafficking.',
                  'Homelessness and displacement due to poverty and urbanization.',
                  'Persistent poverty in certain regions despite overall economic growth.']


In [21]:
issues = ['High youth unemployment despite economic growth.',
          'Poor quality education, especially in rural areas.',
          'Inadequate healthcare access and system.',
          'Women face discrimination and violence.',
          'Widespread corruption negatively impacts economy and society.',
          'Air and water pollution harm health.',
          'Investment needed in roads, railways, airports.',
          'Social discrimination based on caste persists.',
          'Tensions from communal violence between religious groups.',
          'Large population living in poverty.',
          'Security threats from terrorist organizations.',
          'Increasingly sophisticated cyber threats pose challenges.',
          'Wealth inequality among population.',
          'Children forced into dangerous work.',
          'Struggle for rights and protections.',
          'Reform needed for efficiency, transparency, accessibility.',
          'Challenges in housing, infrastructure, and social integration.',
          'Lack of access to digital technologies limits participation.']


In [64]:
for i in os.listdir("sample_images"):
    print(i.split(".")[0])
    process_input_string(i.split(".")[0])
    # break

issues related to child traficing
{'DARPG/P/2021/02415  ': 3.637582, 'DOPPW/E/2018/10796  ': 3.7767093, 'DOWCD/E/2019/02319  ': 3.8536322, 'DARPG/P/2018/05127  ': 3.9622912, 'DOWCD/E/2019/02264  ': 4.761023, 'DOWCD/E/2019/02312  ': 5.008605, 'DOWCD/E/2019/01894  ': 5.463815, 'DOSEL/E/2018/01833  ': 5.709637, 'CDA01/P/2017/00597  ': 5.7280707, 'DOWCD/E/2020/00416  ': 6.287045, 'CGATP/P/2017/01277  ': 6.6834784, 'DOWCD/E/2019/01598  ': 6.943367, 'CMOTR/P/2018/02205  ': 7.0716567, 'DOWCD/E/2019/00002  ': 7.204506, 'DOWCD/E/2019/02835  ': 7.2835193, 'DOPPW/P/2019/06068  ': 7.29518, 'DOWCD/E/2019/01582  ': 7.4033303, 'DOWCD/E/2020/00144  ': 7.4579716, 'CMOTR/P/2018/01658  ': 7.4708786, 'CMOTR/P/2018/02705  ': 7.653984, 'DOWCD/E/2019/02653  ': 7.7072296, 'CGATP/P/2021/01219  ': 7.7252626, 'DOPPW/P/2021/07011  ': 7.725265, 'DOWCD/E/2019/00425  ': 7.8912888, 'JSOLD/E/2018/00119  ': 7.949272, 'DOWCD/E/2019/01548  ': 7.9794307, 'MOPRJ/E/2018/00268  ': 8.042865, 'DARPG/P/2017/05047  ': 8.056734, 

In [ ]:
TOKENIZERS_PARALLELISM = False

In [35]:
final_wording["underprivileed"]

KeyError: 'underprivileed'

In [54]:
df['subject_content'].sample(n=10000).to_csv("allg.csv", index = False)

In [60]:

process_input_string("discrimination against minorities and underprivileged")



{'DARPG/P/2021/03298  ': 3.32279, 'DOURD/P/2019/00445  ': 4.325409, 'MMSME/E/2017/00490  ': 4.7899594, 'DARPG/P/2022/00198  ': 5.2103486, 'DARPG/P/2021/00190  ': 5.210349, 'DARPG/P/2021/03906  ': 5.210349, 'DARPG/P/2020/00756  ': 5.2103515, 'CBOEC/E/2020/01209  ': 5.6404805, 'DARPG/E/2018/23516  ': 5.8021126, 'DARPG/P/2021/00658  ': 5.878457, 'GOVUP/E/2018/14694  ': 6.019045, 'MOIAB/E/2020/03034  ': 6.2494755, 'DARPG/P/2014/02630  ': 6.3667965, 'DARPG/P/2018/00119  ': 6.4959803, 'DARPG/P/2017/01464  ': 6.5177517, 'DARPG/P/2018/02731  ': 6.780686, 'DARPG/P/2016/03245  ': 6.7937775, 'DARPG/P/2016/04507  ': 6.793786, 'DARPG/P/2018/01155  ': 6.8274407, 'DEPOJ/E/2017/01704  ': 6.993677, 'DARPG/P/2017/01919  ': 7.0819235, 'DOFPD/E/2021/01924  ': 7.1118503, 'DARPG/P/2016/05512  ': 7.17297, 'DARPG/E/2016/16392  ': 7.2254305, 'DARPG/P/2020/01254  ': 7.3101134, 'GOVUP/E/2018/10506  ': 7.406602, 'GOVUP/E/2019/13620  ': 7.427233, 'DARPG/P/2019/02127  ': 7.6140695, 'GOVUP/E/2021/60736  ': 7.6502852